# Accidentes de transito INE

## Objetivos

- Como mejorar el data set para prevenir accidentes de transito

### Para lograr el objetivo necesitamos

- Descubrir cuales son las causas de los accidentes de transito
- Identificar que informacion es necesaria para identificar las causas
- Descubrir como se previenen los accidentes de transito

# Causas de los accidentes de transito

Segun aseguradora mapfre las principales causas de accidentes de transito son:
 1. Fallas mecanicas
 2. Conductores ebrios
 3. Sueño
 4. Uso de telefono
 5. Exceso de velocidad
 6. Pasar el semáforo en rojo / Alto
 7. Distracciones
 
Ademas la organizacion mundial de la salud agrega como un factor para los accidentes de transito las infraestructura vial.


https://www.who.int/es/news-room/fact-sheets/detail/road-traffic-injuries 
https://www.mapfre.com.pe/viviendo-en-confianza/seguridad-vial/7-principales-causas-accidentes-auto/


# Datos de investigacion

Datos proveeidos por INE https://www.ine.gob.gt/estadisticasine/index.php/usuario/hechos_transito_menu del año 2009 a 2019

### Que datos se incluyen en los datos de la INE:

- Número de correlativo
- Año de ocurrencia
- Día de ocurrencia
- Hora de ocurrencia
- Grupo de hora
- Grupo de hora mañana, tarde y noche
- Mes de ocurrencia
- Día de la semana de ocurrencia
- Departamento de ocurrencia
- Municipio de ocurrencia
- Zona de ocurrencia
- Tipo de vehículo
- Marca del vehículo
- Color del vehículo
- Modelo del vehículo
- Grupo modelo del vehículo
- Tipo de evento

### Que datos son solicitados por las aseguradoras

- Datos del vehiculo asegurado:
    - Marca del vehiculo
    - Tipo del vehiculo
    - Modelo del vehiculo
    - Color del vehiculo
- Datos del accidente
    - Edad del conductor
    - Fecha
    - Hora
    - Lado de la calle (izquierdo/derecho)
    - Velocidad
    - Direccion en la que se traslada (vehiculos involucrados en el accidente)
    - Causa del accidente
    - Vehiculo que circulaba con preferencia de via

# Limpieza de datos

TODO:
- Juntar todos los años
- Mejorar limpieza

In [54]:
%use krangl

In [49]:
val eventsToClean = DataFrame.readCSV("hechos-transito-2019.csv")

In [9]:
events.head(10)

núm_corre,año_ocu,día_ocu,hora_ocu,g_hora,g_hora_5,mes_ocu,día_sem_ocu,depto_ocu,mupio_ocu,zona_ocu,tipo_veh,marca_veh,color_veh,modelo_veh,g_modelo_veh,tipo_eve
1,2019,1,5,1,1,1,2,1,101,3,4,64,5,2018,5,6
2,2019,1,21,4,3,1,2,1,101,1,2,90,14,9999,99,1
3,2019,1,7,2,1,1,2,1,101,18,1,41,8,2019,5,1
4,2019,1,11,2,1,1,2,1,102,99,1,69,5,2007,4,1
5,2019,1,8,2,1,1,2,1,101,7,4,27,1,9999,99,6
6,2019,1,0,1,1,1,2,1,101,7,1,999,99,9999,99,1
7,2019,1,4,1,1,1,2,1,115,99,4,999,1,9999,99,6
8,2019,1,4,1,1,1,2,1,114,99,1,999,99,9999,99,1
9,2019,1,11,2,1,1,2,1,108,99,1,69,99,9999,99,1
10,2019,1,12,3,2,1,2,1,110,99,1,56,99,9999,99,1


#### Verifiquemos que los datos tengan sentido

In [38]:
eventsToClean["día_ocu"].values().distinct() // No hay fecha con mes con mas de 31 dias o con datos invalidos

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

In [39]:
eventsToClean["mes_ocu"].values().distinct() // No hay mes mayor al 12 (enero 1 ... diciembre 12)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [44]:
eventsToClean["hora_ocu"].values().distinct().sortedBy { it as? Int } // Hora entre 0 y 23 hrs

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

In [45]:
eventsToClean["g_hora_5"].values().distinct() // Sola hay 3 grupos de hora

[1, 3, 2]

In [47]:
eventsToClean["día_sem_ocu"].values().distinct().sortedBy { it as? Int } // Soloa hay 7 dias a la semana (1 ...7 )

[1, 2, 3, 4, 5, 6, 7]

In [50]:
eventsToClean["depto_ocu"].values().distinct().sortedBy { it as? Int } // Solo hay 22 departamentos (1 ... 22)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [52]:
eventsToClean["modelo_veh"].values().distinct().sortedBy { it as? Int } // Los modelos de vehiculos son validos

[1974, 1975, 1978, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 9999]

#### Busquemos datos vacios

In [96]:
// Columnas con valores NA
eventsToClean.cols
    .filter { row -> row.isNA().any { it } }
    .map { it.name }

[]

## Vamos a convertir los codigos en los valores correspondientes

#### Primero definamos los valores por los que se reemplazaran los ids

In [23]:
val occurrenceMonth = dataFrameOf("mes_ocu", "month")(
    1, "Enero",
    2, "Febrero",
    3, "Marzo",
    4, "Abril",
    5, "Mayo",
    6, "Junio",
    7, "Julio",
    8, "Agosto",
    9, "Septiembre",
    10, "Octubre",
    11, "Noviembre",
    12, "Diciembre"
)

In [24]:
val occurrenceDayOfTheWeek = dataFrameOf("día_sem_ocu", "dayOfWeek")(
    1, "Lunes",
    2, "Martes",
    3, "Miércoles",
    4, "Jueves",
    5, "Viernes",
    6, "Sábado",
    7, "Domingo"
)

In [25]:
val eventType = dataFrameOf("tipo_eve", "event")(
    1, "Colisión",
    2, "Choque",
    3, "Vuelco",
    4, "Caída",
    5, "Atropello",
    6, "Derrape",
    7, "Embarranco",
    8, "Encuneto",
    99, "Ignorado"
)

In [30]:
val vehiculeColor = dataFrameOf("color_veh", "vehicule.color")(
    1, "Rojo",
    2, "Blanco",
    3, "Azul",
    4, "Gris",
    5, "Negro",
    6, "Verde",
    7, "Amarillo",
    8, "Celeste",
    9, "Corinto",
    10, "Café",
    11, "Beige",
    12, "Turquesa",
    13, "Marfil",
    14, "Anaranjado",
    15, "Morado",
    16, "Rosado",
    17, "Varios colores",
    99, "Ignorado"
)

In [37]:
val vehiculeType = dataFrameOf("tipo_veh", "vehicule.type")(
    1, "Automóvil",
    2, "Camioneta sport o blazer",
    3, "Pick up",
    4, "Motocicleta",
    5, "Camión",
    6, "Cabezal",
    7, "Bus extraurbano",
    8, "Jeep",
    9, "Microbús",
    10, "Taxi",
    11, "Panel",
    12, "Bus urbano",
    13, "Tractor",
    14, "Moto taxi",
    15, "Furgón",
    16, "Grúa",
    17, "Bus escolar",
    18, "Bicicleta",
    19, "Avioneta",
    20, "Montacargas",
    21, "Bus militar",
    22, "Cuatrimoto",
    23, "Furgoneta",
    99, "Ignorado"
)

In [26]:
val occurrenceDepartment = DataFrame.readCSV("depto_ocu.csv")
    .rename("Occurrence department" to "department")

In [40]:
val towns = DataFrame.readCSV("towns-id.csv")

In [52]:
// Vamos a unir los valores y vamos a renombrar las columnas
val events = eventsToClean
    .innerJoin(occurrenceMonth, by = listOf("mes_ocu"))
    .innerJoin(occurrenceDayOfTheWeek, by = listOf("día_sem_ocu"))
    .innerJoin(eventType, by = listOf("tipo_eve"))
    .innerJoin(vehiculeColor, by = listOf("color_veh"))
    .innerJoin(vehiculeType, by = listOf("tipo_veh"))
    .innerJoin(occurrenceDepartment, by = listOf("depto_ocu"))
    .innerJoin(towns, by = listOf("mupio_ocu"))
    .remove("g_hora_5", "g_hora", "mes_ocu", "día_sem_ocu", "depto_ocu", "g_modelo_veh", 
                "tipo_eve", "color_veh", "marca_veh", "mupio_ocu", "tipo_veh")
    .rename(
        "año_ocu" to "year",
        "día_ocu" to "day",
        "hora_ocu" to "time",
        "modelo_veh" to "vehicule.model",
        "zona_ocu" to "zone",
        "núm_corre" to "correlative"
    )
    
events.head(5)

correlative,year,day,time,zone,vehicule.model,month,dayOfWeek,event,vehicule.color,vehicule.type,department,town
1970,2019,8,20,6,2012,Abril,Lunes,Colisión,Rojo,Automóvil,Guatemala,Guatemala
6032,2019,18,14,13,2015,Noviembre,Lunes,Colisión,Rojo,Automóvil,Guatemala,Guatemala
6643,2019,16,20,9,2012,Diciembre,Lunes,Colisión,Rojo,Automóvil,Guatemala,Guatemala
1497,2019,19,8,1,9999,Marzo,Martes,Colisión,Rojo,Automóvil,Guatemala,Guatemala
1502,2019,19,18,7,9999,Marzo,Martes,Colisión,Rojo,Automóvil,Guatemala,Guatemala


In [91]:
// Para aprovechar el autonaming del IDE vamos a definir variables con los nombres de las columnas
val correlative = "correlative"
val year = "year"
val day = "day"
val time = "time"
val month = "month"
val dayOfWeek = "dayOfWeek"
val zone = "zone"
val vehiculeModel = "vehicule.model"
val vehiculeColor = "vehicule.color"
val vehiculeType= "vehicule.type"
val department = "department"
val town = "town"
val event = "event"

### Comenzemos por ver que mes del año tiene mas accidentes

In [57]:
%use lets-plot

In [87]:
val accidentsPerMonth = events
    .groupBy(month)
    .count()
    .sortedByDescending("n")

In [88]:
val perMonthPlot = mapOf(
    month to accidentsPerMonth[month].asStrings().toList(),
    "count" to accidentsPerMonth["n"].asInts().toList()
)

lets_plot(perMonthPlot) +
    geom_bar(stat = Stat.identity) {
        x = month; y = "count"
    }

### Cual es el tipo de accidente mas comun

In [97]:
val accidentsPerEventType = events
    .groupBy(event)
    .count()
    .sortedByDescending("n")

event,n
Colisión,3623
Atropello,2102
Choque,593
Derrape,407
Vuelco,188
Caída,131
Ignorado,3


In [98]:
val perEventPlot = mapOf(
    event to accidentsPerEventType[event].asStrings().toList(),
    "count" to accidentsPerEventType["n"].asInts().toList()
)

lets_plot(perEventPlot) +
    geom_bar(stat = Stat.identity) {
        x = event; y = "count"
    }